In [1]:
import json
import sys
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from IPython import display

In [5]:
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.topuniversities.com/university-rankings/university-subject-rankings/2022/natural-sciences#university-rankings-indicators")


CSS_SELECTOR = 'div#ranking-data-load a.uni-link'
wait = WebDriverWait(driver, 30)
universities: WebElement = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CSS_SELECTOR)))
# print(elements[1].get_attribute('outerHTML'))
# print(len(elements))


# get the uni links
uni_names = list(map(lambda x: x.text, universities))
# print(uni_names)


uni_links = list(map(lambda x: x.get_attribute('href'), universities))
# print(uni_links)


uni_info = {}

def change_tab(by, value):
    element: WebElement = wait.until(EC.presence_of_element_located((by, value)))
    driver.execute_script("arguments[0].click();", element)  # Working
    sleep(1)




def change_page():
    # next page button
    # XPATH = '//ul[@id="alt-style-pagination"]//li//a[@class="page-link next"]'
    # element: WebElement = wait.until(EC.presence_of_element_located((By.XPATH, XPATH)))
    
    CSS_SELECTOR = 'ul#alt-style-pagination a.page-link.next'

    element: WebElement = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, CSS_SELECTOR)))
    
    
    driver.execute_script("arguments[0].click();", element)  # Working



In [7]:
for indx in range(51):
    while(True):
        try:
            for name,link in zip(uni_names, uni_links):
                print(f"**************  Visiting : {name}  ")
                
                driver.get(link)
                try: 
                    CSS_SELECTOR = "div.tab-content  div.univ-subsection-full-width-value"
                    addmissin_info = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CSS_SELECTOR)))

                    labels = list(map(lambda info:info.find_element(By.CSS_SELECTOR, 'label').text, addmissin_info)) 
                    values = list(map(lambda info:info.find_element(By.CSS_SELECTOR, 'div').text, addmissin_info))
                    
                except Exception as e:
                    print(e)
                
                else: 
                    if uni_info.get(name) is None :
                        uni_info[name] = {"ADDMISSION": {}}

                    uni_info[name]["ADDMISSION"] = {k : v for k,v in zip(labels, values)}

                try:
                    
                    change_tab(By.ID, 'studStaff_Tab')

                    CSS_SELECTOR_TITLE = "div#studStaffTab label.studstaff-subsection-title"
                    titles = list(map(lambda x: x.text , driver.find_elements(By.CSS_SELECTOR ,CSS_SELECTOR_TITLE))) 

                    CSS_SELECTOR_COUNT = "div#studStaffTab div.studstaff-subsection-count"
                    counts = list(map(lambda x: x.text , driver.find_elements(By.CSS_SELECTOR ,CSS_SELECTOR_COUNT)))

                except Exception as e:
                    print(e)

                else: 
                    if uni_info.get(name) is None :
                        uni_info[name] = {"Students and Stuffs": {}}

                    uni_info[name]["Students and Stuffs"] = {k : v for k,v in zip(titles, counts)}
            
            
        except Exception as e:
            print(e)
            continue
        
        else:
            print(len(uni_info))
            if indx != 51 - 1:
                change_page()
                sleep(5) 
            break
            
        print('****************  Visiting completed *********************')

**************  Visiting : Massachusetts Institute of Technology (MIT)  
**************  Visiting : Harvard University  
**************  Visiting : University of Cambridge  
**************  Visiting : University of Oxford  
**************  Visiting : Stanford University  
**************  Visiting : ETH Zurich  
**************  Visiting : California Institute of Technology (Caltech)  
**************  Visiting : University of California, Berkeley (UCB)  
**************  Visiting : Imperial College London  
**************  Visiting : The University of Tokyo  
Message: 

10


TimeoutException: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x00FCF243]
	(No symbol) [0x00F57FD1]
	(No symbol) [0x00E4D04D]
	(No symbol) [0x00E7C0B0]
	(No symbol) [0x00E7C22B]
	(No symbol) [0x00EAE612]
	(No symbol) [0x00E985D4]
	(No symbol) [0x00EAC9EB]
	(No symbol) [0x00E98386]
	(No symbol) [0x00E7163C]
	(No symbol) [0x00E7269D]
	GetHandleVerifier [0x01269A22+2655074]
	GetHandleVerifier [0x0125CA24+2601828]
	GetHandleVerifier [0x01078C0A+619850]
	GetHandleVerifier [0x01077830+614768]
	(No symbol) [0x00F605FC]
	(No symbol) [0x00F65968]
	(No symbol) [0x00F65A55]
	(No symbol) [0x00F7051B]
	BaseThreadInitThunk [0x757E6BD9+25]
	RtlGetFullPathName_UEx [0x77148FD2+1218]
	RtlGetFullPathName_UEx [0x77148F9D+1165]


In [ ]:
print(json.dumps(uni_info, indent = 2))